In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from scipy.sparse import coo_matrix
from sklearn.metrics import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import *
from sklearn.svm import SVC

## Tokens

In [ ]:
# tokens= pd.read_csv("C:\\Users\\jyoth\\Desktop\\Sem 3\\Youtube MOM proj\\Final\\data_corrected.csv")
tokens = pd.read_csv("/content/drive/My Drive/Khatron Ke Khiladi/More data/data_corrected.csv")

In [ ]:
tokens.head()

,Title_token,Desc_token,Tags_token,Category
0,"'sri', 'durga', 'sahasranamam', 'stotram'","'sri', 'durga', 'sahasranamam', 'stotram', 'ch...","'durga', 'sri', 'durga', 'sahasranamam', 'sri'...",Music
1,"'sri', 'maha', 'vishnu', 'sahasranamam', 'sri'...","'sri', 'maha', 'vishnu', 'sahasranamam', 'sri'...","'lakshmi', 'spiriutal', 'sri', 'vishnu', 'saha...",Music
2,"'sri', 'venkateswara', 'sahasranamam'","'sri', 'venkateswara', 'sahasranamam', 'venkat...","'sanskrit', 'venkateswara', 'lord', 'balaji', ...",Music
3,"'sri', 'kubera', 'lakshmi', 'pooja'","'sri', 'kubera', 'lakshmi', 'pooja', 'kubera',...","'kubera', 'sri', 'lakshmi', 'kubera', 'pooja',...",Music
4,"'sri', 'sudarshana', 'lakshmi', 'sahasranamam'","'sri', 'sudarshana', 'lakshmi', 'sahasranamam'...","'sri', 'sudarshana', 'sahasranamam', 'sri', 'l...",Music


In [ ]:
for col in tokens.columns[:-1]:
  tokens[col]=tokens[col].apply(lambda x:str(x).replace("'", "").strip().replace(",", ""))   #Make each field a string of words

In [ ]:
tokens.head()

,Title_token,Desc_token,Tags_token,Category
0,sri durga sahasranamam stotram,sri durga sahasranamam stotram chants thiagara...,durga sri durga sahasranamam sri durga sahasra...,Music
1,sri maha vishnu sahasranamam sri lakshmi dandakam,sri maha vishnu sahasranamam sri lakshmi danda...,lakshmi spiriutal sri vishnu sahasranamam lord...,Music
2,sri venkateswara sahasranamam,sri venkateswara sahasranamam venkateshwara fo...,sanskrit venkateswara lord balaji tirumala tir...,Music
3,sri kubera lakshmi pooja,sri kubera lakshmi pooja kubera sanskrit know ...,kubera sri lakshmi kubera pooja sri lakshmi la...,Music
4,sri sudarshana lakshmi sahasranamam,sri sudarshana lakshmi sahasranamam sudarshana...,sri sudarshana sahasranamam sri lakshmi sahasr...,Music


In [ ]:
tokens.shape

(64771, 4)

In [ ]:
tokens.Category.value_counts()

Automobiles           4745
Music                 4612
Fashion               4579
Food                  4523
Entertainment         4466
News_Politics         4439
Pets_Animals          4406
Science&Technology    4390
Education             4267
Religion              4255
Gaming                4195
Film_and_Animation    4064
Sports                4039
Videoblogging         4027
Travel                3764
Name: Category, dtype: int64

In [ ]:
# tokens.to_csv("C:\\Users\\jyoth\\Desktop\\Sem 3\\Youtube MOM proj\\Final\\tokens.csv", index=False)
tokens.to_csv("/content/drive/My Drive/Khatron Ke Khiladi/More data/Untitled folder/tokens.csv", index=False)

In [ ]:
categories = tokens.Category.unique().tolist()

In [ ]:
title=[]
desc=[]
tags=[]
for cat in categories:
#To get a STRING of keyowrds for each category - THIS
  title.append(' '.join(tokens.Title_token[tokens.Category==cat].tolist()).strip())
  desc.append(' '.join(tokens.Desc_token[tokens.Category==cat].tolist()).strip())
  tags.append(' '.join(tokens.Tags_token[tokens.Category==cat].tolist()).strip())

  # # To get a LIST of keywords for each category
  # # titles.append(tokens.Title_token[tokens.Category==cat].tolist())
  # title.append(list(set([a.strip() for b in tokens.Title_token[tokens.Category==cat].tolist() for a in b]))[1:])
  # desc.append(list(set([a.strip() for b in tokens.Desc_token[tokens.Category==cat].tolist() for a in b]))[1:])
  # tags.append(list(set([a.strip() for b in tokens.Tags_token[tokens.Category==cat].tolist() for a in b]))[1:])


In [ ]:
tokens_cat = pd.DataFrame({"Title":title,
                         "Desc":desc,
                         "Tags":tags,
                        "Category":categories})
tokens_cat

,Title,Desc,Tags,Category
0,sri durga sahasranamam stotram sri maha vishnu...,sri durga sahasranamam stotram chants thiagara...,durga sri durga sahasranamam sri durga sahasra...,Music
1,ghost prank ans entertainment india number gho...,guy bhot jyada mehnat video tyyar hoti hai aga...,ghost prank ans entertainment ans entrtnmnt an...,Entertainment
2,husqvarna svartpilen vitpilen review -the sw...,husqvarna svartpilen husqvarna vitpilen fina...,autocar india autocar india car reviews video ...,Automobiles
3,live information test series quality education...,video description live information test series...,nan average short tricks average short tricks ...,Education
4,nan nan nan nan nan nan nan nan nan nan nan na...,nan nan nan nan nan nan nan happyindependenced...,mahesh babu action movie hindi dub mahesh babu...,Film_and_Animation
5,soft chewy marshmallow treats tasty american f...,chewy soft goodness shape fluffy marshmallow s...,buzzfeed tasty tasty recipe cook cook cook bak...,Food
6,torsten bell rsa bridge future podcast geoff m...,british economy look like coronavirus pandemic...,nan nan nan nan history education bbc mary bea...,News_Politics
7,extend haircut week tip long hair cut right co...,vitaman rmrs hair style click discover vitama...,extend haircut long extend haircut haircut man...,Fashion
8,hilarious emote scam mortal new membership per...,hilarious emote scam mortal new membership per...,soul mortal soul game fall guy funny moment mo...,Gaming
9,best true wireless earbuds pubg mobile cod har...,look good true wireless earbud game wait video...,techmit game earbud mic good earbud game good ...,Videoblogging


In [ ]:
tokens_cat.to_csv("/content/drive/My Drive/Khatron Ke Khiladi/More data/Untitled folder/tokens_cat.csv", index=False)

In [ ]:
tokens.columns[0:3]

Index(['Title_token', 'Desc_token', 'Tags_token'], dtype='object')

In [ ]:
tokens['Text'] = tokens[tokens.columns[0:3]].apply(
    lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [ ]:
tokens['Text'][0]

'sri durga sahasranamam stotram sri durga sahasranamam stotram chants thiagarajan sri durga sahasranamam stotram durga form mother goddess shaktism incarnation goddess durga kali renu renuka bhagvati bhavani ambika lalita gauri kandalini jaya rajeswari appellation skandamata kushmanda shailaputri kaalratri brahmacharini maha gauri katyayani chandraghanta siddhidatri list  name describe popularly use eastern hindus call ashtottara shatanamavali goddess durga subscribe indian classical music channel  goo dhejg durga sridurgasahasranamam durgastotram drrthiagarajan devotional goddessdurga durga sri durga sahasranamam sri durga sahasranamavali maa durga mantra devotional goddess durga durga bhajan durga chalisa durga devi songs sri durga sahasranamam lyrics sahasranamam stotra sanskrit slogan aigiri nandini song durga devi bhakthi song sri durga sahasranamam stotram goddess durga mantra sri durga devi durga devi mantra durga puja goddess navratri songs devi durga maa devi songs durga sahas

In [ ]:
tokens_txt = pd.DataFrame({"Text":tokens.Text,
                        "Category":tokens.Category})
tokens_txt.head()

,Text,Category
0,sri durga sahasranamam stotram sri durga sahas...,Music
1,sri maha vishnu sahasranamam sri lakshmi danda...,Music
2,sri venkateswara sahasranamam sri venkateswara...,Music
3,sri kubera lakshmi pooja sri kubera lakshmi po...,Music
4,sri sudarshana lakshmi sahasranamam sri sudars...,Music


In [ ]:
tokens_txt.shape

(64771, 2)

In [ ]:
tokens_txt.to_csv("/content/drive/My Drive/Khatron Ke Khiladi/More data/Untitled folder/tokens_txt.csv", index=False)

## Features

In [ ]:
# tokens_txt = pd.read_csv("C:\\Users\\jyoth\\Desktop\\Sem 3\\Youtube MOM proj\\Final\\tokens_txt.csv")
tokens_txt = pd.read_csv("/content/drive/My Drive/Khatron Ke Khiladi/More data/Untitled folder/tokens_txt.csv")

In [ ]:
tokens_txt

,Text,Category
0,sri durga sahasranamam stotram sri durga sahas...,Music
1,sri maha vishnu sahasranamam sri lakshmi danda...,Music
2,sri venkateswara sahasranamam sri venkateswara...,Music
3,sri kubera lakshmi pooja sri kubera lakshmi po...,Music
4,sri sudarshana lakshmi sahasranamam sri sudars...,Music
...,...,...
64766,gregorian mass english rev dharmaraj catholic ...,Religion
64767,live holy mass english sebastian church madhav...,Religion
64768,gregorian mass english rev dharmaraj catholic ...,Religion
64769,live english mass rev vineeth george catholich...,Religion


In [ ]:
tokens_txt.Category.value_counts()

Automobiles           4745
Music                 4612
Fashion               4579
Food                  4523
Entertainment         4466
News_Politics         4439
Pets_Animals          4406
Science&Technology    4390
Education             4267
Religion              4255
Gaming                4195
Film_and_Animation    4064
Sports                4039
Videoblogging         4027
Travel                3764
Name: Category, dtype: int64

In [ ]:
tokens_txt.Category.value_counts()/tokens_txt.Category.shape[0]

Automobiles           0.073258
Music                 0.071205
Fashion               0.070695
Food                  0.069831
Entertainment         0.068951
News_Politics         0.068534
Pets_Animals          0.068024
Science&Technology    0.067777
Education             0.065878
Religion              0.065693
Gaming                0.064767
Film_and_Animation    0.062744
Sports                0.062358
Videoblogging         0.062173
Travel                0.058112
Name: Category, dtype: float64

In [ ]:
# tokens_cat = pd.read_csv("C:\\Users\\jyoth\\Desktop\\Sem 3\\Youtube MOM proj\\Final\\tokens_cat.csv")
tokens_cat = pd.read_csv("/content/drive/My Drive/Khatron Ke Khiladi/More data/Untitled folder/tokens_cat.csv")

In [ ]:
tokens_cat['Text'] = tokens_cat[tokens_cat.columns[0:3]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1)

In [ ]:
tokens_cat

,Title,Desc,Tags,Category,Text
0,sri durga sahasranamam stotram sri maha vishnu...,sri durga sahasranamam stotram chants thiagara...,durga sri durga sahasranamam sri durga sahasra...,Music,sri durga sahasranamam stotram sri maha vishnu...
1,ghost prank ans entertainment india number gho...,guy bhot jyada mehnat video tyyar hoti hai aga...,ghost prank ans entertainment ans entrtnmnt an...,Entertainment,ghost prank ans entertainment india number gho...
2,husqvarna svartpilen vitpilen review -the sw...,husqvarna svartpilen husqvarna vitpilen fina...,autocar india autocar india car reviews video ...,Automobiles,husqvarna svartpilen vitpilen review -the sw...
3,live information test series quality education...,video description live information test series...,nan average short tricks average short tricks ...,Education,live information test series quality education...
4,nan nan nan nan nan nan nan nan nan nan nan na...,nan nan nan nan nan nan nan happyindependenced...,mahesh babu action movie hindi dub mahesh babu...,Film_and_Animation,nan nan nan nan nan nan nan nan nan nan nan na...
5,soft chewy marshmallow treats tasty american f...,chewy soft goodness shape fluffy marshmallow s...,buzzfeed tasty tasty recipe cook cook cook bak...,Food,soft chewy marshmallow treats tasty american f...
6,torsten bell rsa bridge future podcast geoff m...,british economy look like coronavirus pandemic...,nan nan nan nan history education bbc mary bea...,News_Politics,torsten bell rsa bridge future podcast geoff m...
7,extend haircut week tip long hair cut right co...,vitaman rmrs hair style click discover vitama...,extend haircut long extend haircut haircut man...,Fashion,extend haircut week tip long hair cut right co...
8,hilarious emote scam mortal new membership per...,hilarious emote scam mortal new membership per...,soul mortal soul game fall guy funny moment mo...,Gaming,hilarious emote scam mortal new membership per...
9,best true wireless earbuds pubg mobile cod har...,look good true wireless earbud game wait video...,techmit game earbud mic good earbud game good ...,Videoblogging,best true wireless earbuds pubg mobile cod har...


In [ ]:
categories = tokens_txt.Category.unique().tolist()
categories

['Music',
 'Entertainment',
 'Automobiles',
 'Education',
 'Film_and_Animation',
 'Food',
 'News_Politics',
 'Fashion',
 'Gaming',
 'Videoblogging',
 'Travel',
 'Sports',
 'Science&Technology',
 'Religion',
 'Pets_Animals']

### TFIDF 

In [ ]:
text_docs = tokens_txt.Text.to_list()
len(text_docs)

64771

In [ ]:
vect = CountVectorizer(max_features=1500, max_df=0.5)
vect.fit(tokens_cat['Text'])
X = vect.transform(tokens_cat['Text'])

In [ ]:
X_final = vect.transform(text_docs)

In [ ]:
#2nd
tf_idf_vector = []
tfidfconv = TfidfTransformer()
tfidf_transformer = tfidfconv.fit(X)
for cat in categories:
    opt = tokens_cat.Title[tokens_cat.Category==cat].iloc[0]
    tf_idf_vector.append(tfidf_transformer.transform(vect.transform([opt])))

In [ ]:
tfidf_vector = pd.DataFrame({'Vector':tf_idf_vector}, index=categories )
tfidf_vector.head()

,Vector
Music,"(0, 1491)\t0.002089150565574779\n (0, 1478)..."
Entertainment,"(0, 1447)\t0.030007096352249282\n (0, 1443)..."
Automobiles,"(0, 1495)\t0.02797261032383623\n (0, 1486)\..."
Education,"(0, 1463)\t0.06311399724954145\n (0, 1462)\..."
Film_and_Animation,"(0, 1485)\t0.011518319546468385\n (0, 1456)..."


In [ ]:
features = vect.get_feature_names()

In [ ]:
#Function for sorting tf_idf in descending order
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

keys_vectors = []
for cat in categories:
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tfidf_vector['Vector'].loc[cat].tocoo())
    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(features,sorted_items,15)
    keys_vectors.append(keywords)
    
# now print the results
    print("\nCategory:", cat)
    print("Keywords:")
    for k,v in keywords.items():
        print(k,v)


Category: Music
Keywords:
mozart 0.354
jukebox 0.263
imman 0.251
muzik 0.243
carnatic 0.235
chopin 0.223
badshah 0.222
beethoven 0.207
anirudh 0.203
devotional 0.201
arijit 0.199
pritam 0.155
vivaldi 0.122
nithyasree 0.122
sivakarthikeyan 0.119

Category: Entertainment
Keywords:
eic 0.513
tvf 0.321
patrol 0.288
lwiay 0.287
vines 0.224
jassi 0.209
jaissi 0.18
peters 0.178
vighnaharta 0.166
lagte 0.152
vaaste 0.149
duje 0.149
aahat 0.142
nahin 0.14
qtiyapa 0.136

Category: Automobiles
Keywords:
powerdrift 0.919
autocar 0.331
eicma 0.091
expo 0.058
suzuki 0.05
honda 0.05
hyundai 0.049
geneva 0.046
skoda 0.045
kawasaki 0.042
ktm 0.038
maruti 0.036
tvs 0.035
benelli 0.035
mahindra 0.035

Category: Education
Keywords:
edureka 0.734
aumsum 0.526
goalcast 0.182
python 0.177
upsc 0.173
hadoop 0.133
ias 0.113
ssc 0.104
byju 0.08
tableau 0.065
webinar 0.063
pmp 0.058
java 0.055
plotly 0.051
ras 0.043

Category: Film_and_Animation
Keywords:
dubbed 0.836
cyanide 0.513
teja 0.069
channelate 0.055
k

In [ ]:
#MAIN Vectorizer
#Directly calc tfidf values from documents without 1st converting them to bag of words
tfidfvect = TfidfVectorizer(max_features=1500, max_df=0.5, use_idf=True)
tfidfvect.fit(tokens_cat['Text'])
tfidf_X = tfidfvect.transform(text_docs).toarray()

In [ ]:
filename = "/content/drive/My Drive/Khatron Ke Khiladi/Final/Models/TFIDF2.sav"
pickle.dump(tfidfvect, open(filename, 'wb'))

In [ ]:
tfidf_DF = pd.DataFrame(tfidf_X, columns = tfidfvect.get_feature_names())
tfidf_DF.head(10)

,aahat,aajtak,aakash,aapko,aapse,aaronmarino,aarti,aayegi,abhijeet,abner,abs,absorption,accessories,acche,accounting,achhe,aci,acog,acp,acquaint,adagio,adesanya,adi,administration,ads,adsense,afcat,affiliates,agaece,agusta,aiim,aiims,aimpoint,aipmt,airbnb,airforce,airtel,ais,aishwarya,ajith,...,wcxzsy,wearing,webdriver,webinar,webonly,websitelearner,wfsvt,wgtdc,whiten,wicket,wildly,wittenberg,wjssv,wolfgang,wpaso,wqpz,wvnv,wweuru,wynk,xgqa,xiaomi,xivkzb,xoxo,xumwf,xuv,xxl,yamaha,yeczfcdzk,yojana,yotuh,ytsync,yuvraj,zane,zanehijazi,zenfone,zigwheels,zrsb,zskgghtnnng,zufbuu,zxiw
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.176251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_DF.shape

(64771, 1500)

In [ ]:
tfidf_DF["wicket"]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
64766    0.0
64767    0.0
64768    0.0
64769    0.0
64770    0.0
Name: wicket, Length: 64771, dtype: float64

## Model Building 

In [ ]:
# categories = tags_data.Category.unique()

In [ ]:
le = LabelEncoder()
le.fit(categories)
le.classes_

array(['Automobiles', 'Education', 'Entertainment', 'Fashion',
       'Film_and_Animation', 'Food', 'Gaming', 'Music', 'News_Politics',
       'Pets_Animals', 'Religion', 'Science&Technology', 'Sports',
       'Travel', 'Videoblogging'], dtype='<U18')

In [ ]:
#Target Variable
y = le.transform(tokens_txt.Category)

In [ ]:
len(y)

64771

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_X, y, test_size=0.3, random_state=0)

### Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, max_samples=0.5)

In [ ]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=0.5,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
filename = '/content/drive/My Drive/Khatron Ke Khiladi/Final/Models/RandomForestModel.sav'
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
rf_pred = rf.predict(X_test)
rf_pred

array([ 7,  3, 10, ..., 14, 10,  0])

In [ ]:
rf_proba = rf.predict_proba(X_test)
rf_proba

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_test,rf_pred))

Confusion Matrix:
 [[ 879    2    2    8   22   12  120    3    1    0    4    8   10  367
     4]
 [   1 1226    0    0    1    0    0    1    0    0    0    0    0   45
     0]
 [   1    0 1304    0    0    2   28    2    1    0    0    0    1   17
     0]
 [   6    1    0 1291    2    0    0    0    0    0    0    0    0   96
     0]
 [  20    0    0    2  877    0    8    5    0    1    1    1   68  201
     0]
 [   1    0    0    0    1 1332    0    0    0    0    0    0    0   21
     0]
 [  41    0   37    0    7    0  981    1    1    0    0    0   10  166
     0]
 [   1    0    0    0    7    0    0 1331    0    3    1    0    2   27
     0]
 [   0    0    0    0    0    0    0    0 1201    1    1    0    0   50
     0]
 [   0    0    0    0    1    6    0    0    0 1283    0    0    1   36
     1]
 [   0    0    0    0    0    0    0    0    0    0 1317    0    0   15
     0]
 [   4    0    0    0    1    0    0    0    0    0    0 1341    0   17
     0]
 [   8    0    0    0

In [ ]:
#With TF-IDF (before splitting; with Weights; With Vectorizer(proper))
print("Precision Score:", precision_score(y_test,rf_pred, average='weighted'))
print("Recall Score:", recall_score(y_test,rf_pred, average='weighted'))
print("F1 Score:", f1_score(y_test,rf_pred, average='weighted'))
print("AUC Score:", roc_auc_score(y_test, rf_proba, multi_class='ovr'))

Precision Score: 0.9338162567467019
Recall Score: 0.9056195965417867
F1 Score: 0.9116464172568502
AUC Score: 0.9945681467344759


In [ ]:
filename = "/content/drive/My Drive/Khatron Ke Khiladi/Final/Models/RandomForestModel.sav"
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(X_test)
print(result)

[ 7  3 10 ... 14 10  0]


### XGBoost

In [ ]:
xgbc = XGBClassifier(objective='multi:softmax')

In [ ]:
xgbc.fit(X_train, y_train)

In [ ]:
# filename = '/content/drive/My Drive/Khatron Ke Khiladi/Final/Models/XGBC2.bin'
# # pickle.dump(xgbc, open(filename, 'wb'))
# xgbc.save_model(filename)

In [ ]:
xgbc_pred = xgbc.predict(X_test)
xgbc_pred

In [ ]:
xgbc_proba = xgbc.predict_proba(X_test)
xgbc_proba

In [ ]:
#With TF-IDF (before splitting; with Weights; With Vectorizer(proper))
print("Confusion Matrix:\n", confusion_matrix(y_test,xgbc_pred))

Confusion Matrix:
 [[1286    1    2    6   17   12   98    1    1    0    2    8    4    0
     4]
 [  70 1202    0    0    1    0    0    0    0    1    0    0    0    0
     0]
 [  33    0 1310    0    1    4    2    1    1    1    0    1    0    2
     0]
 [ 131    0    0 1263    0    0    0    0    0    0    0    0    0    2
     0]
 [ 280    0    0    3  835    0    4    2    0    3    2    0   52    1
     2]
 [  27    0    0    0    0 1328    0    0    0    0    0    0    0    0
     0]
 [ 254    0   43    1    4    0  932    2    1    1    0    0    6    0
     0]
 [  68    0    0    0    3    0    0 1299    0    0    1    1    0    0
     0]
 [  53    0    0    0    0    0    0    0 1197    0    3    0    0    0
     0]
 [  48    0    0    0    1    7    0    0    0 1271    0    0    1    0
     0]
 [  22    0    0    0    0    0    0    0    0    0 1310    0    0    0
     0]
 [  25    0    0    0    0    0    0    0    0    0    0 1336    0    0
     2]
 [ 181    0    0    3

In [ ]:
print("Classification Report:\n", classification_report(y_test,xgbc_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.89      0.59      1442
           1       1.00      0.94      0.97      1274
           2       0.97      0.97      0.97      1356
           3       0.99      0.90      0.95      1396
           4       0.91      0.71      0.80      1184
           5       0.98      0.98      0.98      1355
           6       0.89      0.75      0.82      1244
           7       0.99      0.95      0.97      1372
           8       1.00      0.96      0.98      1253
           9       1.00      0.96      0.98      1328
          10       0.99      0.98      0.99      1332
          11       0.99      0.98      0.99      1363
          12       0.94      0.79      0.86      1176
          13       0.99      0.68      0.81      1116
          14       0.99      0.94      0.97      1241

    accuracy                           0.90     19432
   macro avg       0.94      0.89      0.91     19432
we

In [ ]:
#With TF-IDF (before splitting; with Weights; With Vectorizer(proper))
print("Precision Score:", precision_score(y_test,xgbc_pred, average='weighted'))
print("Recall Score:", recall_score(y_test,xgbc_pred, average='weighted'))
print("F1 Score:", f1_score(y_test,xgbc_pred, average='weighted'))
print("AUC Score:", roc_auc_score(y_test, xgbc_proba, multi_class='ovr'))

Precision Score: 0.9352418839499766
Recall Score: 0.8969226018937835
F1 Score: 0.9069380446271211
AUC Score: 0.9930994838028366


In [ ]:
# filename = "/content/drive/My Drive/Khatron Ke Khiladi/Final/Models/XGradientBoosting.sav"
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.predict(X_test)
# print(result)

### SVC

In [ ]:
svc = SVC(class_weight='balanced', probability=True)

In [ ]:
svc.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
filename = 'SVC1.sav'
pickle.dump(svc, open(filename, 'wb'))

In [ ]:
svc_pred = svc.predict(X_test)

In [ ]:
svc_proba = svc.predict_proba(X_test)

In [ ]:
#With TF-IDF (before splitting; with Weights; With Vectorizer)
print("Confusion Matrix:\n", confusion_matrix(y_test,svc_pred))
print("Precision Score:", precision_score(y_test,svc_pred, average='weighted'))
print("Recall Score:", recall_score(y_test,svc_pred, average='weighted'))
print("F1 Score:", f1_score(y_test,svc_pred, average='weighted'))
print("AUC Score:", roc_auc_score(y_test, svc_proba, multi_class='ovr'))

Confusion Matrix:
 [[1173    1    0    6   39   19  179    1   13    0    2    1    4    2
     2]
 [   5 1262    0    1    0    0    1    0    5    0    0    0    0    0
     0]
 [  14    0 1126    0    1    1  200    2    5    0    0    0    4    2
     1]
 [   7    1    0 1365    0    8    0    0    9    1    0    0    0    5
     0]
 [  65    0    0    1 1005    0    6    1    2    0    1    0   99    4
     0]
 [   3    0    1    0    0 1350    0    0    0    0    0    0    0    1
     0]
 [  92    0   63    0    5    1 1060    2   10    0    0    0   10    0
     1]
 [   0    0    0    0    4    0    0 1363    3    0    0    0    1    1
     0]
 [   5    0    1    0    1    0    0    0 1240    0    0    0    4    2
     0]
 [   4    0    0    1    0    0    0    2    5 1312    1    0    3    0
     0]
 [  16    0    1    0    2    0    0    0    8    0 1304    0    1    0
     0]
 [   3    3    0    0    0    0    0    0    0    0    0 1357    0    0
     0]
 [  28    0    0    1

In [ ]:
filename = "/content/drive/My Drive/Khatron Ke Khiladi/Final/Models/SVC.sav"
loaded_svc = pickle.load(open(filename, 'rb'))
result = loaded_svc.predict(X_test)
print(result)

[ 7  3 10 ... 14 10  0]
